In [1]:
!pip3 install colossalai

     |████████████████████████████████| 256 kB 5.1 MB/s 
     |████████████████████████████████| 195 kB 54.5 MB/s 
     |████████████████████████████████| 224 kB 58.3 MB/s 
     |████████████████████████████████| 98 kB 7.1 MB/s 
     |████████████████████████████████| 8.8 MB 50.9 MB/s 
     |████████████████████████████████| 596 kB 67.3 MB/s 
     |████████████████████████████████| 461 kB 73.4 MB/s 
     |████████████████████████████████| 51 kB 4.6 MB/s 
  Created wheel for colossalai: filename=colossalai-0.1.1-py3-none-any.whl size=284561 sha256=28f952b6509b0c169274e0cc812c31101ebc096962652b583e178fb51cddfce0
  Stored in directory: /root/.cache/pip/wheels/b2/7a/a1/62e6c3722e0305168510c395571827390fc52b93539ab5a196
Successfully built colossalai
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip3 install pytorch-lamb

     |████████████████████████████████| 125 kB 4.5 MB/s 


In [11]:
!pip3 show torch


Name: torch
Version: 1.10.0+cu111
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.7/dist-packages
Requires: typing-extensions
Required-by: torchvision, torchtext, torchaudio, pytorch-lamb, fastai, colossalai


In [3]:
from pytorch_lamb import Lamb

In [4]:
import torch
from torch.optim.optimizer import Optimizer, required


class LARS(Optimizer):
    r"""Implements layer-wise adaptive rate scaling for SGD.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): base learning rate (\gamma_0)
        momentum (float, optional): momentum factor (default: 0) ("m")
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
            ("\beta")
    Based on Algorithm 1 of the following paper by You, Gitman, and Ginsburg.
    Large batch training of convolutional networks with layer-wise adaptive rate scaling. ICLR'18:
        https://openreview.net/pdf?id=rJ4uaX2aW
    The LARS algorithm can be written as
    .. math::
            \begin{aligned}
                v_{t+1} & = \mu * v_{t} + (1.0 - \mu) * (g_{t} + \beta * w_{t}), \\
                w_{t+1} & = w_{t} - lr * ||w_{t}|| / ||v_{t+1}|| * v_{t+1},
            \end{aligned}
    where :math:`w`, :math:`g`, :math:`v` and :math:`\mu` denote the
        parameters, gradient, velocity, and momentum respectively.
    Example:
        >>> optimizer = LARS(model.parameters(), lr=0.1)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    """
    def __init__(self, params, lr=required, momentum=.9,
                 weight_decay=.0005, dampening = 0):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}"
                             .format(weight_decay))
        #if eta < 0.0:
        #    raise ValueError("Invalid eta value:{}".format(eta))

        defaults = dict(lr=lr, momentum = momentum,
                        weight_decay = weight_decay,
                        dampening = dampening)

        super(LARS, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()


        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            lr = group['lr']
            dampening = group['dampening']

            for p in group['params']:
                if p.grad is None:
                    continue

                param_state = self.state[p]
                # gradient
                d_p = p.grad.data
                weight_norm = torch.norm(p.data)

                # update the velocity
                if 'momentum_buffer' not in param_state:
                    buf = param_state['momentum_buffer'] = torch.zeros_like(p.data)
                else:
                    buf = param_state['momentum_buffer']
                # l2 regularization
                if weight_decay != 0:
                    d_p.add_(p, alpha=weight_decay)

                buf.mul_(momentum).add_(d_p, alpha = 1.0 - dampening)
                v_norm = torch.norm(buf)

                local_lr = lr * weight_norm / (1e-6 + v_norm)

                # Update the weight
                p.add_(buf, alpha = -local_lr)


        return loss

In [5]:
import os
from pathlib import Path
import math
import colossalai
import torch
import torch.nn as nn
import torch.nn.functional as F
from colossalai.core import global_context as gpc
from colossalai.logging import get_dist_logger
from colossalai.nn import CosineAnnealingLR
from colossalai.nn.metric import Accuracy
from colossalai.trainer import Trainer, hooks
from colossalai.utils import MultiTimer, get_dataloader
from torchvision import transforms
from torchvision.datasets import MNIST
from tqdm import tqdm



class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits


config = {'BATCH_SIZE':128,'NUM_EPOCHS':30}

colossalai.launch(config=config,rank=0,world_size=1,host='127.0.0.1',port=1234)

logger = get_dist_logger()

# build 

model = LeNet5(n_classes=10)

# build dataloaders
train_dataset = MNIST(
    root=Path('./tmp/'),
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

test_dataset = MNIST(
    root=Path('./tmp/'),
    train=False,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

train_dataloader = get_dataloader(dataset=train_dataset,
                                  shuffle=True,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

test_dataloader = get_dataloader(dataset=test_dataset,
                                  add_sampler=False,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

# build criterion
criterion = torch.nn.CrossEntropyLoss()

# optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=5e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
# optimizer = Lamb(model.parameters(), lr=0.1, weight_decay=5e-4, betas=(.9, .999), adam=True)
# optimizer = Lamb(model.parameters(), lr=0.1, weight_decay=5e-4, betas=(.9, .999), adam=False)
# optimizer = LARS(model.parameters(), lr=0.1, weight_decay=5e-4)


#exponentially increase learning rate from low to high
def lrs(batch):
    low = math.log2(1e-5)
    high = math.log2(10)
    return 2**(low+(high-low)*batch/len(train_dataloader)/gpc.config.NUM_EPOCHS)

# lr_scheduler
# lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lrs)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, math.log2(10), steps_per_epoch=2, epochs=30)
# lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 10, 15, 20, 25])
# lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20])

engine, train_dataloader, test_dataloader, _ = colossalai.initialize(model,
                                                                      optimizer,
                                                                      criterion,
                                                                      train_dataloader,
                                                                      test_dataloader,
                                                                      )
# build a timer to measure time
timer = MultiTimer()

# create a trainer object
trainer = Trainer(
    engine=engine,
    timer=timer,
    logger=logger
)

# define the hooks to attach to the trainer
hook_list = [
    hooks.LossHook(),
    hooks.LRSchedulerHook(lr_scheduler=lr_scheduler, by_epoch=False),
    # hooks.AccuracyHook(accuracy_func=Accuracy()),
    hooks.LogMetricByEpochHook(logger),
    hooks.LogMemoryByEpochHook(logger),
    hooks.LogTimingByEpochHook(timer, logger),

    # you can uncomment these lines if you wish to use them
    hooks.TensorboardHook(log_dir='./tb_logs', ranks=[0]),
    # hooks.SaveCheckpointHook(checkpoint_dir='./ckpt')
]

# start training
trainer.fit(
    train_dataloader=train_dataloader,
    epochs=gpc.config.NUM_EPOCHS,
    test_dataloader=test_dataloader,
    test_interval=1,
    hooks=hook_list,
    display_progress=True
)

If you want to activate cuda mode for MoE, please install with cuda_ext!
Colossalai should be built with cuda extension to use the FP16 optimizer


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


RuntimeError: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs